# Data Preparation


In [1]:
import numpy as np
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile

## Obtaining data
Data is published at https://www.citibikenyc.com/system-data  
Monthly archives.

In [6]:
d = "Source data/data/nyc"
download_path = os.path.join(d, 'raw')
parsed = os.path.join(d , "parsed")

P.S do not run below code because it will start downloading the 8Gb from the webpage

In [7]:
soup = BeautifulSoup(open(os.path.join(d, "data.html")), 'html.parser')
for link in soup.findAll('a'):
    url = link.get('href')
    name = url.rsplit('/', 1)[-1]
    path = os.path.join(download_path, name)
    if name.endswith(".zip") and not os.path.exists(path):
        r = requests.get(url, allow_redirects=True)
        open(path, 'wb').write(r.content)


KeyboardInterrupt: 

Calculate count by day.  
Some files have "Start Time" and others have "starttime".  
Concatenating DFs creates an unwieldy DF. Easier to parse data month by month and save grouped output.

In [9]:
def parse_tripdata(path):
    zip_file = ZipFile(path)
    for file in zip_file.infolist():
        if "__MACOSX" not in file.filename:
            df = pd.read_csv(zip_file.open(file.filename))
            if "Start Time" in df.columns:
                df.rename(columns = {"Start Time": "starttime"}, inplace=True)
            elif "started_at" in df.columns:
                df.rename(columns = {"started_at": "starttime"}, inplace=True)
            df["starttime"] = pd.to_datetime(df["starttime"])
            df_date = df[["starttime"]].groupby(by = df["starttime"].dt.date).count().rename(columns={'starttime':'count'}).reset_index()
    return df_date

In [10]:
for path in os.listdir(download_path):
    if path.endswith(".zip"):
        full_path = os.path.join(download_path, path)
        out_path = os.path.join(parsed, path + ".csv")
        if not os.path.exists(out_path):
            print(path)
            df_parsed = parse_tripdata(full_path)
            df_parsed.to_csv(os.path.join(parsed, path + ".csv"))

In [13]:
dfs= []

for path in os.listdir(parsed):
    full_path = os.path.join(parsed, path)
    df = pd.read_csv(full_path, index_col=0)
    dfs.append(df)
    
df_nyc = pd.concat(dfs)
df_nyc.rename(columns = {"starttime": "date"}, inplace=True)
df_nyc.sort_values(by="date", inplace=True)

# Group stray values that got into a wrong month
df_nyc = df_nyc.groupby("date").sum().reset_index()
df_nyc["date"] = pd.to_datetime(df_nyc["date"])
blank = pd.read_csv(r"Source data/data/nyc/blank_dates.csv",sep=",")
blank["Date"] = pd.to_datetime(blank["Date"])
df_nyc = pd.merge(blank, df_nyc, how ="left", left_on = "Date", right_on = "date")
df_nyc = df_nyc.drop(["Date"],axis = 1)
df_nyc.fillna(method='ffill', inplace=True)
df_nyc

,date,count
0,2014-01-01,6059.0
1,2014-02-01,12771.0
2,2014-03-01,9202.0
3,2014-04-01,23908.0
4,2014-05-01,26762.0
...,...,...
2917,2021-12-27,32525.0
2918,2021-12-28,41862.0
2919,2021-12-29,40747.0
2920,2021-12-30,48208.0


In [ ]:
# To add some 

In [14]:
df_nyc["Year"] = pd.DatetimeIndex(df_nyc["date"]).year
df_nyc["Month"] = pd.DatetimeIndex(df_nyc["date"]).month
df_nyc["Week"] = pd.DatetimeIndex(df_nyc["date"]).week
df_nyc["Day"] = pd.DatetimeIndex(df_nyc["date"]).day
df_nyc["Day of Week"] = pd.DatetimeIndex(df_nyc["date"]).dayofweek
df_nyc["Day of Year"] = pd.DatetimeIndex(df_nyc["date"]).dayofyear

C:\Users\Admin\AppData\Local\Temp\ipykernel_36136\3947180096.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df_nyc["Week"] = pd.DatetimeIndex(df_nyc["date"]).week


In [18]:
df_nyc.head(20)

,date,count,Year,Month,Week,Day,Day of Week,Day of Year
0,2014-01-01,6059.0,2014,1,1,1,2,1
1,2014-02-01,12771.0,2014,2,5,1,5,32
2,2014-03-01,9202.0,2014,3,9,1,5,60
3,2014-04-01,23908.0,2014,4,14,1,1,91
4,2014-05-01,26762.0,2014,5,18,1,3,121
5,2014-06-01,31333.0,2014,6,22,1,6,152
6,2014-07-01,34854.0,2014,7,27,1,1,182
7,2014-08-01,32654.0,2014,8,31,1,4,213
8,2014-09-01,23575.0,2014,9,36,1,0,244
9,2014-10-01,27942.0,2014,10,40,1,2,274


# Saving file

#### Note: we copied this file manually and transfered for further exploration in data_preparation_and_exploration and models

In [ ]:
df_nyc.to_csv(os.path.join(d, "nyc-daily.csv"))